In [ ]:
import os
import shutil
import tarfile
import tensorflow as tf
import pandas as pd
from bs4 import BeautifulSoup
import re
import sys
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go
from wordcloud import WordCloud, STOPWORDS
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import keras
from keras.callbacks import EarlyStopping
from scipy.special import softmax
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from transformers import BertTokenizer, TFBertModel, TFBertForSequenceClassification, BatchEncoding, BertConfig
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import tf_keras
!pip install datasets
import datasets
from datasets import load_dataset
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#RUN ONLY IF YOU WANT THE SST-2 DATASET (1)
sst2_dataset = load_dataset("glue", "sst2")
train_dataset = sst2_dataset["train"]
validation_dataset = sst2_dataset["validation"]
test_dataset = sst2_dataset["test"]
train_df=pd.DataFrame(train_dataset.to_pandas())
val_df=pd.DataFrame(validation_dataset.to_pandas())
test_df=pd.DataFrame(test_dataset.to_pandas())
df=pd.concat([train_df, val_df], ignore_index=True)
df.drop(columns=['idx'], inplace=True)
print(df)

In [ ]:
#RUN ONLY IF YOU WANT THE SST-2 DATASET (2)
train_df, test_df=train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
val_df, test_df=train_test_split(test_df, test_size=0.5, stratify=test_df['label'], random_state=42)
train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
train_df, unlab_df= train_test_split(train_df, test_size=0.4, stratify=train_df['label'], random_state=42)
unlab_df.loc[:, 'label']=2
print(train_df)
print(unlab_df)
print(val_df)
print(test_df)
#For the first split, put the frac value below to be equal to 0.0031 for about 100 labeled samples
#For the second split, put the frac value below to be equal to 0.01 for about 300 labeled samples
#For the third split, put the frac value below to be equal to 0.0153 for about 500 labeled samples
#For the fourth split, put the frac value below to be equal to 0.031 for about 1000 labeled samples
#For the fifth split put the frac value below to be equal to 0.0765 for about 2500 labeled samples
#For the sixth split, put the frac value below to be equal to 0.153 for about 5000 labeled samples
train_df=train_df.groupby('label', group_keys=False).apply(lambda x: x.sample(frac=0.153))
print(len(train_df))
print(len(unlab_df))
print(len(val_df))
print(len(test_df))

In [ ]:
#RUN ONLY IF YOU WANT THE Coronavirus NLP tweets DATASET (1)
from google.colab import files
uploaded = files.upload()

In [ ]:
#RUN ONLY IF YOU WANT THE Coronavirus NLP tweets DATASET (2)
train_df=pd.read_csv('/content/Corona_NLP_train.csv', encoding='ISO-8859-1')
test_df=pd.read_csv('/content/Corona_NLP_test.csv', encoding='ISO-8859-1')
train_df=train_df.drop(['UserName', 'ScreenName', 'Location', 'TweetAt'], axis=1)
test_df=test_df.drop(['UserName', 'ScreenName', 'Location', 'TweetAt'], axis=1)
train_df.drop_duplicates(subset='OriginalTweet',inplace=True)
test_df.drop_duplicates(subset='OriginalTweet',inplace=True)
df=pd.concat([train_df, test_df], axis=0)
train_df, test_df=train_test_split(df, test_size=0.2, stratify=df['Sentiment'], random_state=42)
val_df, test_df=train_test_split(test_df, test_size=0.5, stratify=test_df['Sentiment'], random_state=42)
mapping = {'Extremely Negative': 0, 'Negative': 1, 'Neutral': 2, 'Positive':3, 'Extremely Positive':4}
train_df['Sentiment']=train_df['Sentiment'].replace(mapping)
val_df['Sentiment']=val_df['Sentiment'].replace(mapping)
test_df['Sentiment']=test_df['Sentiment'].replace(mapping)
train_df['Sentiment']=train_df['Sentiment'].astype('category')
val_df['Sentiment']=val_df['Sentiment'].astype('category')
test_df['Sentiment']=test_df['Sentiment'].astype('category')
train_df, unlab_df= train_test_split(train_df, test_size=0.42, stratify=train_df['label'], random_state=42)
unlab_df['Sentiment']=unlab_df['Sentiment'].cat.add_categories(5)
unlab_df['Sentiment']=5
#For the first split, put the frac value below to be equal to 0.0051 for about 100 labeled
#For the second split, put the frac value below to be equal to 0.0255 for about 500 labeled
#For the third split, put the frac value below to be equal to 0.051 for about 1000 labeled
#For the fourth split, put the frac value below to be equal to 0.2545 for about 5000 labeled
train_df=train_df.groupby('Sentiment', group_keys=False).apply(lambda x: x.sample(frac=0.0051))
print(len(train_df))
print(len(unlab_df))
print(len(val_df))
print(len(test_df))

In [ ]:
#RUN ONLY IF YOU WANT SST-5 DATASET
dataset=load_dataset("SetFit/sst5")
train_dataset=dataset['train']
val_dataset=dataset['validation']
test_dataset=dataset['test']
train_df=pd.DataFrame(train_dataset)
train_df=train_df.drop(columns=['label_text'])
val_df=pd.DataFrame(val_dataset)
val_df=val_df.drop(columns=['label_text'])
test_df=pd.DataFrame(test_dataset)
test_df=test_df.drop(columns=['label_text'])
train_df, unlab_df= train_test_split(train_df, test_size=0.53, stratify=train_df['label'], random_state=42)
unlab_df.loc[:, 'label']=5
#For the first split, put the frac value below to be equal to 0.027 for about 100 labeled
#For the second split, put the frac value below to be equal to 0.125 for about 500 labeled
#For the third split, put the frac value below to be equal to 0.25 for about 1000 labeled
#For the fourth split, put the frac value below to be equal to 0.5 for about 5000 labeled
train_df=train_df.groupby('label', group_keys=False).apply(lambda x: x.sample(frac=0.125))
train_df.drop_duplicates(subset='text',inplace=True)
unlab_df.drop_duplicates(subset='text', inplace=True)
val_df.drop_duplicates(subset='text',inplace=True)
test_df.drop_duplicates(subset='text',inplace=True)
print(len(train_df))
print(len(unlab_df))
print(len(val_df))
print(len(test_df))

In [ ]:
#RUN ONLY IF YOU WANT THE IMDB Movie Review DATASET
cwd=os.getcwd()
data=tf.keras.utils.get_file(fname ="aclImdb.tar.gz", origin="https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", cache_dir= cwd, extract = True)
data_path=os.path.join(os.path.dirname(data), 'aclImdb')
os.listdir(data_path)
train_dir=os.path.join(data_path, 'train')
for file in os.listdir(train_dir):
    file_path=os.path.join(train_dir, file)
    if os.path.isfile(file_path):
        with open(file_path) as f:
            first_line=f.readline().strip()
            print(file + ":" + first_line)
def load_dataset(dir, typeOfData):
    dictnry = {"review": [], "sentiment": []}
    if typeOfData == 'labelled':
        pos_path=os.path.join(dir, 'pos')
        neg_path=os.path.join(dir, 'neg')
        for file in os.listdir(pos_path):
            file_path=os.path.join(pos_path, file)
            with open(file_path, 'r') as f:
                dictnry["review"].append(f.read())
                dictnry["sentiment"].append(1)
        for file in os.listdir(neg_path):
            file_path=os.path.join(neg_path, file)
            with open(file_path, 'r') as f:
                dictnry["review"].append(f.read())
                dictnry["sentiment"].append(0)
    elif typeOfData == 'unsup':
        unsup_path=os.path.join (dir, 'unsup')
        for file in os.listdir(unsup_path):
            file_path=os.path.join(unsup_path, file)
            with open(file_path, 'r') as f:
                dictnry["review"].append(f.read())
                dictnry["sentiment"].append(2)
    return dictnry
Train_df=pd.DataFrame.from_dict(load_dataset(train_dir, 'labelled'))
Unlab_df=pd.DataFrame.from_dict(load_dataset(train_dir, 'unsup'))
test_dir=os.path.join(data_path, 'test')
Test_df=pd.DataFrame.from_dict(load_dataset(test_dir, 'labelled'))
df=pd.concat([Train_df, Test_df], axis=0)
train_df, test_df=train_test_split(df, test_size=0.2, stratify=df['sentiment'], random_state=42)
val_df, test_df=train_test_split(test_df, test_size=0.5, stratify=test_df['sentiment'], random_state=42)
train_df, unlab_df= train_test_split(train_df, test_size=0.375, stratify=train_df['sentiment'], random_state=42)
unlab_df.loc[:, 'sentiment']=2
#For the first split, put the frac value from the train_df below to be equal to 0.004 for about 100 labeled
#For the second split, put the frac value from the train_df below to be equal to 0.012 for about 300 labeled
#For the third split, put the frac value from the train_df below to be equal to 0.04 for about 1000 labeled
#For the fourth split, put the frac value from the train_df below to be equal to 0.2 for about 5000 labeled
train_df=train_df.groupby('sentiment', group_keys=False).apply(lambda x: x.sample(frac=0.2))
val_df=val_df.groupby('sentiment', group_keys=False).apply(lambda x: x.sample(frac=1))
test_df=test_df.groupby('sentiment', group_keys=False).apply(lambda x: x.sample(frac=1))
print(len(train_df))
print(len(unlab_df))
print(len(val_df))
print(len(test_df))

In [ ]:
class clusterThenLabelKMeans(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.tz=BertTokenizer.from_pretrained('bert-base-uncased')
        self.bert_model=TFBertModel.from_pretrained('bert-base-uncased', num_attention_heads=12)
        self.bertClassification=TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

    def cleaning(self, text):
        soup=BeautifulSoup(text, "html.parser")
        text=re.sub(r"\[[^]]*\]" , '', soup.get_text())
        text=re.sub(r"[^A-Za-z0-9\s,']", '', text)
        return text

    def get_embeddings(self, text, label, typeOfData, batch_size, max_len=64):
        if typeOfData=='train':
            df=train_df
        elif typeOfData=='unlab':
            df=unlab_df
        elif typeOfData=='val':
            df=val_df
        elif typeOfData=='test':
            df=test_df
        df['cleaned']=df[text].apply(self.cleaning)
        targets=df[label]
        tokens=self.tz.batch_encode_plus(df['cleaned'].tolist(), padding='max_length', truncation=True, max_length=max_len, return_tensors='tf')
        dataset=tf.data.Dataset.from_tensor_slices((tokens['input_ids'], tokens['attention_mask']))
        dataset=dataset.batch(batch_size)
        embeddings=[]
        for batch in dataset:
            embeddings.append(self.bert_model(batch[0], batch[1])['last_hidden_state'])
        concatenated_embeddings=[]
        with tf.device("/CPU:0"):
            for i in range(0, len(embeddings), batch_size):
                batch=embeddings[i:i+batch_size]
                concatenated_batch=tf.concat(batch, axis=0)
                concatenated_embeddings.append(concatenated_batch)
            embeddings=tf.concat(concatenated_embeddings, axis=0)
        targets=tf.convert_to_tensor(targets)
        return tokens, embeddings[:,0], targets

    def choosing_num_clusters(self, data):
        #FOR A NUMBER OF CLUSTERS YOU WANT, YOU SUBTRACT TWO. SO IF YOU WANT 2 CLUSTERS, YOU PUT THE VALUE 0 IN num_clusters
        num_clusters=1
        return num_clusters

    def compile(self, optimizer, loss, metrics):
        super().compile()
        self.loss=loss
        self.opt=optimizer
        self.mtrc=metrics
        return self.loss, self.opt, self.mtrc

    def clusteringKMeans(self, labeled, unlabeled, batch_size):
        self.numLabeled=labeled[1].shape[0]
        self.numUnlabeled=unlabeled[1].shape[0]
        with tf.device("/CPU:0"):
            labeled_input_ids_batched = tf.concat(list(tf.data.Dataset.from_tensor_slices(labeled[0]['input_ids']).batch(batch_size)), axis=0)
            labeled_attention_mask_batched = tf.concat(list(tf.data.Dataset.from_tensor_slices(labeled[0]['attention_mask']).batch(batch_size)), axis=0)
            labeled_fullEmbeddedData_batched = tf.concat(list(tf.data.Dataset.from_tensor_slices(labeled[1]).batch(batch_size)), axis=0)
            labeled_fullTargets_batched = tf.concat(list(tf.data.Dataset.from_tensor_slices(labeled[2]).batch(batch_size)), axis=0)
            unlabeled_input_ids_batched = tf.concat(list(tf.data.Dataset.from_tensor_slices(unlabeled[0]['input_ids']).batch(batch_size)), axis=0)
            unlabeled_attention_mask_batched = tf.concat(list(tf.data.Dataset.from_tensor_slices(unlabeled[0]['attention_mask']).batch(batch_size)), axis=0)
            unlabeled_fullEmbeddedData_batched = tf.concat(list(tf.data.Dataset.from_tensor_slices(unlabeled[1]).batch(batch_size)), axis=0)
            unlabeled_fullTargets_batched = tf.concat(list(tf.data.Dataset.from_tensor_slices(unlabeled[2]).batch(batch_size)), axis=0)
            inputIds=tf.concat([labeled_input_ids_batched,  unlabeled_input_ids_batched], axis=0)
            attentionMask=tf.concat([labeled_attention_mask_batched, unlabeled_attention_mask_batched], axis=0)
            fullEmbeddedData=tf.concat([labeled_fullEmbeddedData_batched, unlabeled_fullEmbeddedData_batched], axis=0)
            fullTargets=tf.concat([labeled_fullTargets_batched, unlabeled_fullTargets_batched], axis=0)
        k=self.choosing_num_clusters(fullEmbeddedData)
        kmeans=KMeans(n_clusters=k+2, random_state=42, n_init='auto').fit(fullEmbeddedData)
        num_clusters=k+2
        return kmeans.labels_, num_clusters, inputIds, attentionMask, fullTargets

    def organizingClusters(self, cluster_labels, num_clusters, input_ids, attention_mask, allTargets):
        clusterIndices=[]
        clustersOGLabels=[]
        clusterInputIds=[]
        clusterAttentionMask=[]
        labeledClusterIndices=[]
        unlabeledClusterIndices=[]
        labeledClusterInputIds=[]
        labeledClusterAttentionMask=[]
        labeledClusterLabels=[]
        unlabeledClusterInputIds=[]
        unlabeledClusterAttentionMask=[]
        unlabeledClusterLabels=[]
        num_classes=train_df[train_df.columns[1]].nunique()
        possible_cluster_labels=[i for i in range(num_clusters)]
        for i in possible_cluster_labels:
            clusterIndices.append(np.where(cluster_labels==i)[0].tolist())
            clustersOGLabels.append(tf.gather(allTargets, clusterIndices[i]))
            labeledClusterIndices.append(np.where(clustersOGLabels[i]!=num_classes)[0].tolist())
            unlabeledClusterIndices.append(np.where(clustersOGLabels[i]==num_classes)[0].tolist())
            clusterInputIds.append(tf.gather(input_ids, clusterIndices[i]))
            clusterAttentionMask.append(tf.gather(attention_mask, clusterIndices[i]))
            labeledClusterInputIds.append(tf.gather(clusterInputIds[i], labeledClusterIndices[i]))
            labeledClusterAttentionMask.append(tf.gather(clusterAttentionMask[i], labeledClusterIndices[i]))
            labeledClusterLabels.append(tf.gather(clustersOGLabels[i], labeledClusterIndices[i]))
            unlabeledClusterInputIds.append(tf.gather(clusterInputIds[i], unlabeledClusterIndices[i]))
            unlabeledClusterAttentionMask.append(tf.gather(clusterAttentionMask[i], unlabeledClusterIndices[i]))
            unlabeledClusterLabels.append(tf.gather(clustersOGLabels[i], unlabeledClusterIndices[i]))
        return clusterIndices, clusterInputIds, clusterAttentionMask, clustersOGLabels, labeledClusterInputIds, labeledClusterAttentionMask, labeledClusterLabels, unlabeledClusterInputIds, unlabeledClusterAttentionMask, unlabeledClusterLabels

In [ ]:
ex=clusterThenLabelKMeans()

In [ ]:
train=ex.get_embeddings(train_df.columns[0], train_df.columns[1], 'train', 32)
print("DONE")
unlabeled=ex.get_embeddings(unlab_df.columns[0], unlab_df.columns[1], 'unlab', 32)
print("DONE")
val=ex.get_embeddings(val_df.columns[0], val_df.columns[1], 'val', 32)
print("DONE")
test=ex.get_embeddings(test_df.columns[0], test_df.columns[1], 'test', 32)
unique, _=tf.unique(train[2])
num_classes=unique.shape[0]

In [ ]:
A=ex.clusteringKMeans(train, unlabeled, 4)

In [ ]:
B=ex.organizingClusters(A[0], A[1], A[2], A[3], A[4])

In [ ]:
if num_classes == 2:
    loss=tf_keras.losses.BinaryCrossentropy(from_logits=True)
    metric=tf_keras.metrics.BinaryAccuracy('accuracy')
elif num_classes == 5:
    loss=tf_keras.losses.CategoricalCrossentropy(from_logits=True)
    metric=tf_keras.metrics.CategoricalAccuracy('accuracy')
#YOU SET THE PATIENCE VALUE HERE
callback=tf_keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

In [ ]:
valY=tf.one_hot(val[2], depth=num_classes)
testY=tf.one_hot(test[2], depth=num_classes)
history=[]
trainY=[]
unlabPseudoLabels=[]
initial_lr=1e-5
for i in range(A[1]):
    if i == 0:
        opt=tf_keras.optimizers.legacy.Adam(learning_rate=initial_lr)
    elif i == 1:
        opt=tf_keras.optimizers.legacy.Adam(learning_rate=initial_lr)
    print(i)
    trainY.append(tf.one_hot(B[6][i], depth=num_classes))
    num_epochs=100
    config=BertConfig.from_pretrained('bert-base-uncased')
    config.num_labels=num_classes
    config.hidden_dropout_prob=0.3
    model=TFBertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)
    model.compile(optimizer=opt, loss=loss, metrics=[metric])
    history.append(model.fit([B[4][i], B[5][i]], trainY[i], batch_size=32, epochs=num_epochs, verbose=1, shuffle=True, callbacks=[callback], validation_data=([val[0]['input_ids'], val[0]['attention_mask']], valY)))
    predY=tf.convert_to_tensor(model.predict([B[7][i], B[8][i]])[0])
    if num_classes == 2:
        unlabPseudoLabels.append(tf.argmax(tf.nn.sigmoid(predY), axis=1))
    elif num_classes == 5:
        unlabPseudoLabels.append(tf.argmax(tf.nn.softmax(predY, axis=1), axis=1))
    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")

In [ ]:
dataInputIds=[]
dataAttentionMask=[]
dataY=[]
with tf.device("/CPU:0"):
    for i in range(A[1]):
        dataInputIds.append(tf.concat([B[4][i], B[7][i]], axis=0))
        dataAttentionMask.append(tf.concat([B[5][i], B[8][i]], axis=0))
        dataY.append(tf.concat([B[6][i], unlabPseudoLabels[i]], axis=0))
    dataInputIds=tf.concat(dataInputIds, axis=0)
    dataAttentionMask=tf.concat(dataAttentionMask, axis=0)
    dataY=tf.one_hot(tf.concat(dataY, axis=0), depth=num_classes)

In [ ]:
initial_lr=8e-6
opt=tf_keras.optimizers.legacy.Adam(learning_rate=initial_lr)
if num_classes == 2:
    loss=tf_keras.losses.BinaryCrossentropy(from_logits=True)
    metric=tf_keras.metrics.BinaryAccuracy('accuracy')
elif num_classes == 5:
    loss=tf_keras.losses.CategoricalCrossentropy(from_logits=True)
    metric=tf_keras.metrics.CategoricalAccuracy('accuracy')
callback=tf_keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
num_epochs=100
config=BertConfig.from_pretrained('bert-base-uncased')
config.num_labels=num_classes
config.hidden_dropout_prob=0.2
modelFull=TFBertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)
modelFull.compile(optimizer=opt, loss=loss, metrics=[metric])
historyFull=modelFull.fit([dataInputIds, dataAttentionMask], dataY, batch_size=32, epochs=num_epochs, verbose=1, shuffle=True, callbacks=[callback], validation_data=([val[0]['input_ids'], val[0]['attention_mask']], valY))

In [ ]:
test_loss, test_accuracy = modelFull.evaluate([test[0]['input_ids'], test[0]['attention_mask']], testY)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')

In [ ]:
sup_trainY=tf.one_hot(train[2], depth=num_classes)
sup_valY=tf.one_hot(val[2], depth=num_classes)
sup_testY=tf.one_hot(test[2], depth=num_classes)

In [ ]:
#HERE YOU RUN THE SUPERVISED MODEL
num_epochs=100
config=BertConfig.from_pretrained('bert-base-uncased')
config.hidden_dropout_prob=0.3
config.num_labels=num_classes
model=TFBertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)
initial_lr=3e-5
#lr_schedule = tf_keras.optimizers.schedules.ExponentialDecay(initial_lr, decay_steps=500, decay_rate=0.9)
opt=tf_keras.optimizers.legacy.Adam(learning_rate=initial_lr)
if num_classes == 2:
    loss=tf_keras.losses.BinaryCrossentropy(from_logits=True)
    metric=tf_keras.metrics.BinaryAccuracy('accuracy')
elif num_classes == 5:
    loss=tf_keras.losses.CategoricalCrossentropy(from_logits=True)
    metric=tf_keras.metrics.CategoricalAccuracy('accuracy')
callback=tf_keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=5e-4, patience=7, restore_best_weights=True)
#reduce_lr=tf_keras.callbacks.ReduceLROnPlateau(monitor='val_loss', mode="min", factor=0.7, patience=1, min_lr=2e-6)
model.compile(optimizer=opt, loss=loss, metrics=[metric])
history=model.fit([train[0]['input_ids'], train[0]['attention_mask']], sup_trainY, batch_size=32, epochs=num_epochs, shuffle=True, callbacks=[callback], verbose=1, validation_data=([val[0]['input_ids'], val[0]['attention_mask']], sup_valY))

In [ ]:
test_loss, test_accuracy = model.evaluate([test[0]['input_ids'], test[0]['attention_mask']], sup_testY)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')